# League of Legends Classification
**Author**: Eric Wehmueller
***

## Overview

This project is the third project for Flatiron School's bootcamp program in Data Science.  We are being placed into a hypothetical situation as a Data Scientist and hoping to provide value to our business for the scenario we are given.

## Business Problem

I have been hired by a real estate agency that helps homeowners sell homes.  For this project, I am to provide expected/estimated home prices to homeowners based on the logistics of their home.  This can also give insight on how home renovations might increase the estimated value of their homes, and what type of potential renovations are best.

## Overview
This project is the second project for Flatiron School's bootcamp program in Data Science. We are being placed into a hypothetical situation as a Data Scientist and hoping to provide value to our business for the scenario we are given.

## Business Problem
I have been hired by a real estate agency that helps homeowners sell homes. For this project, I am to provide expected/estimated home prices to homeowners based on the logistics of their home. This can also give insight on how home renovations might increase the estimated value of their homes, and what type of potential renovations are best.